In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [ ]:
N_centr = 5
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="input/"
file_names=["my-m_ee_Run14AuAu_77th_new_19818_760runs.root"] 
#m_ee_Run14AuAu_81st_new_19831_100runs (hadrons)
#m_ee_Run14AuAu_79th_new_19821_1043runs m_ee_Run14AuAu_80th_new_19825_300runs (no vtx)
#m_ee_Run14AuAu_76th_new_19812_75runs m_ee_Run14AuAu_77th_new_19818_760runs #m_ee_Run14AuAu_78th_new_19820_500runs no vtx
#m_ee_Run14AuAu_74th_new_19784_984runs m_ee_Run14AuAu_75th_new_19794_741runs #hadrons
#m_ee_Run14AuAu_70th_new_19721_1065runs m_ee_Run14AuAu_73rd_new_19777_990runs
#m_ee_Run14AuAu_61st_new_19692_328runs m_ee_Run14AuAu_73rd_new_19777_100runs
#m_ee_Run14AuAu_58th_new_19680_1017runs m_ee_Run14AuAu_60th_new_19686_1016runs
#m_ee_Run14AuAu_50th_new_19640_1072runs m_ee_Run14AuAu_57th_new_19676_1072runs
#m_ee_Run14AuAu_48th_new_19631_387runs m_ee_Run14AuAu_49th_new_19636_1072runs
#m_ee_Run14AuAu_46th_new_19619_999runs test_BG_v94_0 m_ee_Run14AuAu_47th_new_19623_1078runs 
sim_file_path="../sim/output/Newembed/sngl/"
primary_file_names=['phi_50M_v9.root']
hadron_file_names=['charged_pi_100M_v1.root']
dalitz_file_names=['dalitz_50M_v12.root']
conv_file_names=['photon_100M_v2.root']
ccbar_name = ['ccbar_50M_v5.root']
bbbar_name = ['bbbar_10M_v0.root']
sim_file_names = [dalitz_file_names[0], primary_file_names[0], conv_file_names[0], ccbar_name[0], bbbar_name[0]]
legend_names = ["real","#pi^{0}","#eta","#gamma #rightarrow ee","c#bar{c}","b#bar{b}"] 
hist_select_3D_names = ["DCA2_hist_0","DCA2_hist_1","DCA2_hist_2","DCA2_hist_3","DCA2_hist_4","sDCA2_hist_0","sDCA2_hist_1","sDCA2_hist_2","sDCA2_hist_3","sDCA2_hist_4",\
                        "myvtx_hist_0","myvtx_hist_1","myvtx_hist_2",'myvtx_hist_3','myvtx_hist_4']

In [3]:
N_hists = len(hist_select_3D_names)
loc = True
Save_to_html = False

In [4]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

input/my-m_ee_Run14AuAu_77th_new_19818_760runs.root
../sim/output/Newembed/sngl/dalitz_50M_v12.root
../sim/output/Newembed/sngl/phi_50M_v9.root
../sim/output/Newembed/sngl/photon_100M_v2.root
../sim/output/Newembed/sngl/ccbar_50M_v5.root
../sim/output/Newembed/sngl/bbbar_10M_v0.root


In [5]:

conv_type = [1,25]
pt = [0.45,0.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    dalitz_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(dalitz_proj,Lcolor=4, Msize=0)
    phi_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"phi_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(phi_proj,Lcolor=root.kGreen+2)
    conv_proj = hists_sim[2][type_loc].ProjectionX(hists_sim[2][type_loc].GetName()+f"conv_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2)
    ccbar_proj = hists_sim[3][type_loc].ProjectionX(hists_sim[3][type_loc].GetName()+f"ccbar_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=root.kMagenta, Msize=0)
    bbbar_proj = hists_sim[4][type_loc].ProjectionX(hists_sim[4][type_loc].GetName()+f"bbbar_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(bbbar_proj,Lcolor=root.kOrange+7)
    real_proj.Scale(1./(1+real_proj.Integral()))
    dalitz_proj.Scale(1./(1+dalitz_proj.Integral()))
    phi_proj.Scale(1./(1+phi_proj.Integral()))
    conv_proj.Scale(1./(1+conv_proj.Integral()))
    ccbar_proj.Scale(1./(1+ccbar_proj.Integral()))
    bbbar_proj.Scale(1./(1+bbbar_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    real_proj.Draw()
    #dalitz_proj.Draw("sameH")
    #phi_proj.Draw("same")
    conv_proj.Draw("sameH")
    #ccbar_proj.Draw("sameH")
    #bbbar_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(dalitz_proj,legend_names[1],"l")
    legends[-1].AddEntry(phi_proj,legend_names[2],"l")
    legends[-1].AddEntry(conv_proj,legend_names[3],"l")
    legends[-1].AddEntry(ccbar_proj,legend_names[4],"l")
    legends[-1].AddEntry(bbbar_proj,legend_names[5],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    print(f"Standard deviation for dalitz_proj in centrality bin {icentr}: {dalitz_proj.GetStdDev()}")
    #print(f"Standard deviation for phi_proj in centrality bin {icentr}: {phi_proj.GetStdDev()}")
    #print(f"Standard deviation for conv_proj in centrality bin {icentr}: {conv_proj.GetStdDev()}")
    print(f"Standard deviation for ccbar_proj in centrality bin {icentr}: {ccbar_proj.GetStdDev()}")
    #print(f"Standard deviation for bbbar_proj in centrality bin {icentr}: {bbbar_proj.GetStdDev()}")

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

Standard deviation for real_proj in centrality bin 0: 175.47960064754403
Standard deviation for dalitz_proj in centrality bin 0: 140.36248687176897
Standard deviation for ccbar_proj in centrality bin 0: 187.79852320876185
Standard deviation for real_proj in centrality bin 1: 167.08399743506044
Standard deviation for dalitz_proj in centrality bin 1: 133.6085943861778
Standard deviation for ccbar_proj in centrality bin 1: 187.07854606587827
Standard deviation for real_proj in centrality bin 2: 174.3747640212405
Standard deviation for dalitz_proj in centrality bin 2: 128.2081324829358
Standard deviation for ccbar_proj in centrality bin 2: 187.22482430234294
Standard deviation for real_proj in centrality bin 3: 176.27176826059394
Standard deviation for dalitz_proj in centrality bin 3: 118.59478478411513
Standard deviation for ccbar_proj in centrality bin 3: 185.0079174131342
Standard deviation for real_proj in centrality bin 4: 194.55300351860998
Standard deviation for dalitz_proj in centr

In [6]:

conv_type = [1,25]
pt = [0.45,0.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
counts = []
keffs = [[1000,0.2,0.9,0.05,0.2],[1000,0.2,0.9,0.08,0.05],[1000,0.2,0.9,0.04,0.1],[1000,0.2,0.9,0.02,0.1],[1000,0.,0.,0.01,0.0]]
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=4, Msize=1.5)
    sum_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sum_proj,Lcolor=1,Msize=0, Lwidth=4)
    phi_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"phi1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(phi_proj,Lcolor=root.kGreen+2)
    conv_proj = hists_sim[2][type_loc].ProjectionX(hists_sim[2][type_loc].GetName()+f"conv1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2)
    ccbar_proj = hists_sim[3][type_loc].ProjectionX(hists_sim[3][type_loc].GetName()+f"ccbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=root.kMagenta)
    bbbar_proj = hists_sim[4][type_loc].ProjectionX(hists_sim[4][type_loc].GetName()+f"bbbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(bbbar_proj,Lcolor=root.kOrange+7)
    counts.append([sum_proj.Integral(),phi_proj.Integral(),conv_proj.Integral(),ccbar_proj.Integral(),bbbar_proj.Integral()])
    #print(round(sum_proj.Integral()*0.2,-1),round(phi_proj.Integral()*0,-1),round(conv_proj.Integral()*0.9,-1),round(ccbar_proj.Integral()*0.02,-1),round(bbbar_proj.Integral()*0.01,-1))
    sum_proj.Add(sum_proj,keffs[icentr][0]-1.)
    sum_proj.Add(phi_proj,keffs[icentr][1])
    sum_proj.Add(conv_proj,keffs[icentr][2])
    sum_proj.Add(ccbar_proj,keffs[icentr][3])
    sum_proj.Add(bbbar_proj,keffs[icentr][4])
    real_proj.Scale(1./(1+real_proj.Integral()))
    sum_proj.Scale(1./(1+sum_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    real_proj.Draw("P")
    sum_proj.Draw("same H")
    legends.append(root.TLegend(0.2,0.7,0.8,0.95))
    legends[-1].SetNColumns(2)
    legends[-1].AddEntry(real_proj,legend_names[0],"p")
    legends[-1].AddEntry(sum_proj,"sum sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.85,0.95,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [7]:
decomposition = []
for icentr in range(N_centr):
    local_sum = [counts[icentr][i]*keffs[icentr][i] for i in range(5)]
    local_sum = sum(local_sum)
    decomposition.append([round(counts[icentr][i]*keffs[icentr][i]/local_sum,2) for i in range(5)])
print(decomposition)

[[0.95, 0.0, 0.0, 0.05, 0.0], [0.91, 0.0, 0.0, 0.09, 0.0], [0.95, 0.0, 0.0, 0.04, 0.0], [0.93, 0.02, 0.02, 0.02, 0.02], [0.99, 0.0, 0.0, 0.01, 0.0]]


In [8]:

conv_type = [1,25]
pt = [2.45,2.65]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
counts1 = []
keffs1 = [[40000,0.05,0.9,0.05,0.2],[10000,0.05,0.9,0.04,0.15],[10000,0.05,0.9,0.04,0.1],[10000,0.05,0.9,0.02,0.1],[10000,0.05,0.9,0.01,0.05]]
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=4)
    sum_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sum_proj,Lcolor=1,Msize=0, Lwidth=4)
    phi_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"phi1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(phi_proj,Lcolor=root.kGreen+2)
    conv_proj = hists_sim[2][type_loc].ProjectionX(hists_sim[2][type_loc].GetName()+f"conv1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2)
    ccbar_proj = hists_sim[3][type_loc].ProjectionX(hists_sim[3][type_loc].GetName()+f"ccbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=root.kMagenta)
    bbbar_proj = hists_sim[4][type_loc].ProjectionX(hists_sim[4][type_loc].GetName()+f"bbbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(bbbar_proj,Lcolor=root.kOrange+7)
    counts1.append([sum_proj.Integral(),phi_proj.Integral(),conv_proj.Integral(),ccbar_proj.Integral(),bbbar_proj.Integral()])
    #print(round(sum_proj.Integral()*0.2,-1),round(phi_proj.Integral()*0,-1),round(conv_proj.Integral()*0.9,-1),round(ccbar_proj.Integral()*0.02,-1),round(bbbar_proj.Integral()*0.01,-1))
    sum_proj.Add(sum_proj,keffs1[icentr][0]-1.)
    sum_proj.Add(phi_proj,keffs1[icentr][1])
    sum_proj.Add(conv_proj,keffs1[icentr][2])
    sum_proj.Add(ccbar_proj,keffs1[icentr][3])
    sum_proj.Add(bbbar_proj,keffs1[icentr][4])
    real_proj.Scale(1./(1+real_proj.Integral()))
    sum_proj.Scale(1./(1+sum_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-500,500)
    real_proj.GetYaxis().SetRangeUser(0.0001,10)
    real_proj.Draw("P")
    sum_proj.Draw("same H")
    legends.append(root.TLegend(0.2,0.7,0.8,0.95))
    legends[-1].SetNColumns(2)
    legends[-1].AddEntry(real_proj,legend_names[0],"p")
    legends[-1].AddEntry(sum_proj,"sum sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.85,0.95,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    print(f"Standard deviation for dalitz_proj in centrality bin {icentr}: {dalitz_proj.GetStdDev()}")

if not Save_to_html: c0.Draw()
if not Save_to_html: c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

Standard deviation for real_proj in centrality bin 0: 86.76956396133866
Standard deviation for dalitz_proj in centrality bin 0: 119.50670013706623
Standard deviation for real_proj in centrality bin 1: 93.71448400804714
Standard deviation for dalitz_proj in centrality bin 1: 119.50670013706623
Standard deviation for real_proj in centrality bin 2: 108.83266839865642
Standard deviation for dalitz_proj in centrality bin 2: 119.50670013706623
Standard deviation for real_proj in centrality bin 3: 110.41695698698922
Standard deviation for dalitz_proj in centrality bin 3: 119.50670013706623
Standard deviation for real_proj in centrality bin 4: 123.9938354219981
Standard deviation for dalitz_proj in centrality bin 4: 119.50670013706623


In [9]:
decomposition1 = []
for icentr in range(N_centr):
    local_sum = [counts1[icentr][i]*keffs1[icentr][i] for i in range(5)]
    local_sum = sum(local_sum)
    decomposition1.append([round(counts1[icentr][i]*keffs1[icentr][i]/local_sum,2) for i in range(5)])
print(decomposition1)

[[0.93, 0.0, 0.0, 0.05, 0.01], [0.76, 0.02, 0.02, 0.16, 0.04], [0.65, 0.05, 0.08, 0.14, 0.08], [0.32, 0.15, 0.24, 0.03, 0.26], [0.13, 0.26, 0.39, 0.01, 0.21]]


In [10]:

conv_type = [25,25]
pt = [0.45,0.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
counts2 = []
keffs2 = [[4000,0.05,0.9,0.05,0.2],[4000,0.05,0.9,0.04,0.15],[0.4,0.05,0.9,0.04,0.1],[0.4,0.05,0.9,0.02,0.1],[0.4,0.05,0.9,0.01,0.05]]
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+5
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=4)
    sum_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sum_proj,Lcolor=1,Msize=0, Lwidth=4)
    phi_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"phi1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(phi_proj,Lcolor=root.kGreen+2)
    conv_proj = hists_sim[2][type_loc].ProjectionX(hists_sim[2][type_loc].GetName()+f"conv1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2)
    ccbar_proj = hists_sim[3][type_loc].ProjectionX(hists_sim[3][type_loc].GetName()+f"ccbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=root.kMagenta)
    bbbar_proj = hists_sim[4][type_loc].ProjectionX(hists_sim[4][type_loc].GetName()+f"bbbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(bbbar_proj,Lcolor=root.kOrange+7)
    counts1.append([sum_proj.Integral(),phi_proj.Integral(),conv_proj.Integral(),ccbar_proj.Integral(),bbbar_proj.Integral()])
    #print(round(sum_proj.Integral()*0.2,-1),round(phi_proj.Integral()*0,-1),round(conv_proj.Integral()*0.9,-1),round(ccbar_proj.Integral()*0.02,-1),round(bbbar_proj.Integral()*0.01,-1))
    sum_proj.Add(sum_proj,keffs2[icentr][0]-1.)
    sum_proj.Add(phi_proj,keffs2[icentr][1])
    sum_proj.Add(conv_proj,keffs2[icentr][2])
    sum_proj.Add(ccbar_proj,keffs2[icentr][3])
    sum_proj.Add(bbbar_proj,keffs2[icentr][4])
    real_proj.Scale(1./(1+real_proj.Integral()))
    sum_proj.Scale(1./(1+sum_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-500,500)
    real_proj.GetYaxis().SetRangeUser(0.0001,10)
    real_proj.Draw("P")
    sum_proj.Draw("same H")
    legends.append(root.TLegend(0.2,0.7,0.8,0.95))
    legends[-1].SetNColumns(2)
    legends[-1].AddEntry(real_proj,legend_names[0],"p")
    legends[-1].AddEntry(sum_proj,"sum sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.85,0.95,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()

if not Save_to_html: c0.Draw()
if not Save_to_html: c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [11]:

conv_type = [25,25]
pt = [0.95,0.95]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+5
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    dalitz_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(dalitz_proj,Lcolor=4)
    phi_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"phi_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(phi_proj,Lcolor=root.kGreen+2)
    conv_proj = hists_sim[2][type_loc].ProjectionX(hists_sim[2][type_loc+4].GetName()+f"conv_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2)
    ccbar_proj = hists_sim[3][type_loc].ProjectionX(hists_sim[3][type_loc+4].GetName()+f"ccbar_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=root.kMagenta)
    bbbar_proj = hists_sim[4][type_loc].ProjectionX(hists_sim[4][type_loc+4].GetName()+f"bbbar_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(bbbar_proj,Lcolor=root.kOrange+7)
    real_proj.Scale(1./(1+real_proj.Integral()))
    dalitz_proj.Scale(1./(1+dalitz_proj.Integral()))
    phi_proj.Scale(1./(1+phi_proj.Integral()))
    conv_proj.Scale(1./(1+conv_proj.Integral()))
    ccbar_proj.Scale(1./(1+ccbar_proj.Integral()))
    bbbar_proj.Scale(1./(1+bbbar_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-700,700)
    real_proj.GetYaxis().SetRangeUser(0.0001,10)
    real_proj.Draw()
    dalitz_proj.Draw("sameH")
    phi_proj.Draw("same")
    conv_proj.Draw("sameH")
    ccbar_proj.Draw("sameH")
    bbbar_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(dalitz_proj,legend_names[1],"l")
    legends[-1].AddEntry(phi_proj,legend_names[2],"l")
    legends[-1].AddEntry(conv_proj,legend_names[3],"l")
    legends[-1].AddEntry(ccbar_proj,legend_names[4],"l")
    legends[-1].AddEntry(bbbar_proj,legend_names[5],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()

if not Save_to_html: c0.Draw()
if not Save_to_html: c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [12]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
legends = []
proj_y = hists_read[0][10].ProjectionY(hists_read[0][0].GetName()+f"pos_proj_y",1,-1,1,-1)
biny0 = proj_y.FindBin(-0.02)
biny1 = proj_y.FindBin(0.09)
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+10
    pos_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"pos_proj_{icentr}",biny0,biny1,1,-1)
    Format_Hist_total(pos_proj,title_x="x, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    #pos_proj.Fit("gaus","R")
    legends.append(root.TLegend(0.5,0.6,0.95,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(pos_proj.Integral(pos_proj.FindBin(0.27),pos_proj.FindBin(0.4))/pos_proj.Integral())
    print(f"Standard deviation for {icentr}: {pos_proj.GetStdDev()*10000:.0f} mum")

if not Save_to_html: c0.Draw()
if not Save_to_html: c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

0.9999996919123547
Standard deviation for 0: 140 mum
0.9999996968235179
Standard deviation for 1: 141 mum
0.9999986819627715
Standard deviation for 2: 142 mum
0.9999530688967486
Standard deviation for 3: 162 mum
0.9998043658050731
Standard deviation for 4: 183 mum


In [13]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
proj_x = hists_read[0][8].ProjectionX(hists_read[0][0].GetName()+f"pos_proj_x",1,-1,1,-1)
biny0 = proj_x.FindBin(0.27)
biny1 = proj_x.FindBin(0.40)
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+10
    pos_proj = hists_read[0][type_loc].ProjectionY(hists_read[0][type_loc].GetName()+f"pos_proj_{icentr}",1,-1,1,-1)
    Format_Hist_total(pos_proj,title_x="y, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(pos_proj.Integral(pos_proj.FindBin(-0.02),pos_proj.FindBin(0.09))/pos_proj.Integral())
    print(f"Standard deviation for {icentr}: {pos_proj.GetStdDev()*10000:.0f} mum")

if not Save_to_html: c0.Draw()
if not Save_to_html: c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

0.9999999547816588
Standard deviation for 0: 93 mum
0.999999955689593
Standard deviation for 1: 93 mum
0.9999997213193339
Standard deviation for 2: 94 mum
0.9999964114738229
Standard deviation for 3: 109 mum
0.9999735788474999
Standard deviation for 4: 121 mum


In [14]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+10
    pos_proj = hists_read[0][type_loc].ProjectionZ(hists_read[0][type_loc].GetName()+f"pos_proj_{icentr}",1,-1,1,-1)
    Format_Hist_total(pos_proj,title_x="z, cm",offset_y=1.1)
    #root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

if not Save_to_html: c0.Draw()
if not Save_to_html: c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")